In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import torch
# print(torch.cuda.is_available())  # Should print True
# print(torch.cuda.get_device_name(0))  # Should print 'T4'

In [ ]:
# Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import re
from collections import defaultdict, Counter
import math

In [3]:
device = torch.device("cuda")

In [3]:
data1 = pd.read_csv('/content/drive/MyDrive/CSV_New_Vector/Data_CSV_100_500/50MB.csv')
# data2 = pd.read_csv('/content/drive/MyDrive/CSV_New_Vector/data2.csv', header=None)
# data3 = pd.read_csv('/content/drive/MyDrive/CSV_New_Vector/data3.csv')
# data4 = pd.read_csv('/content/drive/MyDrive/CSV_New_Vector/data4.csv')
# class_0 = pd.read_csv('/content/drive/MyDrive/csv/accident.csv', index_col=[0])
# texts = class_0['news_articles'].dropna().tolist()

In [ ]:
# data1 = pd.read_csv("your_file.csv", header=None)
# data2.columns = ["news_articles"]  # Set column names manually

In [ ]:
# print(data1.columns)
# print(data1.shape)

In [ ]:
# news = pd.concat([data1, data2, data3, data4])

In [4]:
data1.head()

,news_articles
0,भारतीय अंतरिक्ष अनुसंधान संगठन (इसरो) ने चंद्र...
1,ब्लू होल समुद्र के नीचे बड़ी गुफाएं/गड्ढे या स...
2,नासा सन ऐंड स्पेस के आधिकारिक X हैंडल ने सूर्य...
3,नासा द्वारा दुनिया की करीब 30 लाख नदियों पर कि...
4,वैज्ञानिकों ने एक निएंडरथल महिला का चेहरा री-क...


In [5]:
texts = data1['news_articles'].dropna().tolist()

# **Preprocessing and Tokenization**

In [6]:
# Custom stop words and tokenizer
stop_words = set(['।','',' ',' ','!','! ','!  ','! !','! ! ','! ! !','?','अरे', 'चैनल्स', 'ये','ही','तुमसे','बार','आप','तुम्हारे','तु','रहा','कुछ','कभी',
              'एक','तुम','होता','नहीं','कितनी','पर','तू','हो','है','क्यों','एप','कर','काम','रहे','बातें','लग','आता','ये चैनल्स','करनी','अपना','पैक्स','चीज़','क्या','अरे ये','करा','मैं',
              'अत','अपना','अपनी','अपने','अभी','अंदर','आदि','आप','इत्यादि','इन','इनका','इन्हीं','इन्हें','इन्हों','इस','इसका','इसकी','इसके','इसमें',
              'इसी','इसे','उन','उनका','उनकी','उनके','उनको','उन्हीं','उन्हें','उन्हों','उस','उसके','उसी','उसे','एक','एवं','एस','ऐसे','और','कई',
              'कर','करता','करते','करना','करने','करें','कहते','कहा','का','काफ़ी','कि','कितना','किन्हें','किन्हों','किया','किर','किस','किसी','किसे','की',
              'कुछ','कुल','के','को','कोई','कौन','कौन','बही','बहुत','बाद','बाला','बिलकुल','भी','भीतर','मगर','मानो','मे','में','यदि','यह','यहाँ','यही',
              'या','यिह','ये','रखें','रहा','रहे','ऱ्वासा','लिए','लिये','लेकिन','व','वग़ैरह','वर्ग','वह','वहाँ','वहीं','वाले','वुह','वे','सकता','सकते','सबसे',
              'सभी','साथ','साबुत','साभ','सारा','से','सो','संग','ही','हुआ','हुई','हुए','है','हैं','हो','होता','होती','होते','होना','होने'])


In [7]:
def preprocess(text):
    # Convert to lowercase
    # text = text.lower()

    # # Remove single quote, double quote, full stop
    text = re.sub(r"[.'\"]", "", text)
    # Remove English characters, special characters, and new lines
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)  # Keeping only Hindi characters and spaces
    #remove numbers
    text= re.sub(r"\d+", "", text)
    text = re.sub(r'http[s]?://\S+', '', text)  # Remove hyperlinks
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F]', '', text)  # Remove non-printable characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and new lines
    # Tokenize (split into words)
    words = text.split()

    # Remove stopwords
    filtered_words = [word for word in words if word not in stop_words]

    return filtered_words  # Return as a list of tokens

# Process the entire dataset
tokenized_sentences = [preprocess(text) for text in texts]

In [ ]:
# print(tokenized_sentences)

# **Vocabulary Mapping**

In [8]:
word_counts = Counter(word for sentence in tokenized_sentences for word in sentence)
vocab = list(word_counts.keys())
vocab_size = len(vocab)
word_to_id = {word: i for i, word in enumerate(vocab)}
id_to_word = {i: word for word, i in word_to_id.items()}

print(f"Vocabulary Size: {vocab_size}")

Vocabulary Size: 56123


***Removing min occurence words***

In [33]:
# # Set minimum occurrence threshold
# min_occurrence = 3

# # Filter vocabulary (Keep high-frequency words)
# vocab = {word: idx for word, idx in word_to_id.items() if word_counts[word] > min_occurrence}

# # Identify extra stop words (Low-frequency words)
# extra_stop_words = [word for word in word_to_id if word_counts[word] <= min_occurrence]

# # Save extra stop words to a text file
# with open("/content/drive/MyDrive/DummyFolder/extra_stop_words_100MB.txt", "w", encoding="utf-8") as f:
#     for word in extra_stop_words:
#         f.write(word + "\n")

# print(f"Filtered/Actual Vocabulary Size: {len(vocab)}")
# print(f"Extra Stop Words Saved ({len(extra_stop_words)} words) in extra_stop_words.txt")

Filtered/Actual Vocabulary Size: 29300
Extra Stop Words Saved (26823 words) in extra_stop_words.txt


In [ ]:
# # Load extra stop words from the text file and convert them into a set
# with open("/content/drive/MyDrive/DummyFolder/extra_stop_words_100MB.txt", "r", encoding="utf-8") as f:
#     extra_stop_words = set(f.read().splitlines())

# # Verify the set
# print(f"Loaded {len(extra_stop_words)} extra stop words.")

In [ ]:
# # Combine both sets
# combined_stop_words = stop_words.union(extra_stop_words)
# print(f"Total stop words: {len(combined_stop_words)}")


In [ ]:
# def remove_extra_stopwords(text):
#     # Convert to lowercase
#     # text = text.lower()

#     # # Remove single quote, double quote, full stop
#     text = re.sub(r"[.'\"]", "", text)
#     # Remove English characters, special characters, and new lines
#     text = re.sub(r'[^\u0900-\u097F\s]', '', text)  # Keeping only Hindi characters and spaces
#     #remove numbers
#     text= re.sub(r"\d+", "", text)
#     text = re.sub(r'http[s]?://\S+', '', text)  # Remove hyperlinks
#     text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F]', '', text)  # Remove non-printable characters
#     text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and new lines
#     # Tokenize (split into words)
#     words = text.split()

#     # Remove stopwords
#     filtered_words = [word for word in words if word not in combined_stop_words]

#     return filtered_words  # Return as a list of tokens

# # Example: Process the entire dataset
# tokenized_actual_sentences = [remove_extra_stopwords(text) for text in texts]

In [ ]:
# actual_word_counts = Counter(word for sentence in tokenized_actual_sentences for word in sentence)
# actual_vocab = list(actual_word_counts.keys())
# actual_vocab_size = len(actual_vocab)
# actual_word_to_id = {word: i for i, word in enumerate(actual_vocab)}
# actual_id_to_word = {i: word for word, i in actual_word_to_id.items()}

In [ ]:
# word_to_id = {word: i for i, word in enumerate(vocab)}
# id_to_word = {i: word for word, i in word_to_id.items()}


In [ ]:
# print(f"Vocabulary Size: {actual_vocab_size}")

In [9]:
total_length_import = sum(len(text.split()) for text in texts)
print(f"Total Length at Import: {total_length_import}")


Total Length at Import: 3939982


In [10]:
total_length_before = sum(len(sentence) for sentence in tokenized_sentences)
print(f"Total Length After Pre-Processing: {total_length_before}")

Total Length After Pre-Processing: 2535219


In [ ]:
# total_length_after = sum(len(sentence) for sentence in tokenized_actual_sentences)
# print(f"Total Length After Filtering: {total_length_after}")


In [11]:
reduction_import_to_tokenized = ((total_length_import - total_length_before) / total_length_import) * 100
# reduction_tokenized_to_filtered = ((total_length_before - total_length_after) / total_length_before) * 100
# reduction_import_to_filtered = ((total_length_import - total_length_after) / total_length_import) * 100

print(f"Reduction from Import after Pre-Processing: {reduction_import_to_tokenized:.2f}%")
# print(f"Reduction from Tokenized to Pre-processing: {reduction_tokenized_to_filtered:.2f}%")
# print(f"Total Reduction from Import to Filtered: {reduction_import_to_filtered:.2f}%")


Reduction from Import after Pre-Processing: 35.65%


# **Cooccurrence Matrix using Distance Based Weighting**

In [ ]:
# # Convert sentences to ID arrays
# import torch
# import numpy as np
# import scipy.sparse
# token_ids = [[word_to_id[word] for word in sent] for sent in tokenized_sentences]

# # Convert to PyTorch GPU tensors
# token_ids = [torch.tensor(sent, dtype=torch.int32, device=device) for sent in token_ids]

# # Vocabulary size
# vocab_size = len(word_to_id)

# # Window size
# window_size = 8

# # File path for storage
# save_path = "/content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MBNEW.npz"

# # Initialize storage variables
# word_ids_list, context_ids_list, values_list = [], [], []
# batch_size = 10000  # Adjust based on available memory

# def compute_cooccurrence(token_ids):
#     """Compute co-occurrence matrix using a sliding window approach."""
#     global word_ids_list, context_ids_list, values_list

#     for idx, sentence in enumerate(token_ids):
#         sent_len = sentence.shape[0]
#         for i in range(sent_len):
#             word_id = sentence[i].item()
#             start = max(0, i - window_size)
#             end = min(sent_len, i + window_size + 1)

#             for j in range(start, end):
#                 if i != j:
#                     context_id = sentence[j].item()
#                     distance = abs(j - i)
#                     increment = 1.0 / distance
#                     word_ids_list.append(word_id)
#                     context_ids_list.append(context_id)
#                     values_list.append(increment)

#             # Save data when reaching batch size
#             if len(word_ids_list) >= batch_size:
#                 save_to_disk()

#         # Periodic flushing to avoid memory overflow
#         if idx % 1000 == 0:
#             save_to_disk()

#     # Final save
#     save_to_disk()

# import os

# def save_to_disk():
#     """Append co-occurrence data to an existing sparse matrix instead of overwriting."""
#     global word_ids_list, context_ids_list, values_list

#     if word_ids_list:
#         # Convert current batch to sparse matrix
#         new_matrix = scipy.sparse.coo_matrix(
#             (values_list, (word_ids_list, context_ids_list)),
#             shape=(vocab_size, vocab_size)
#         )

#         if os.path.exists(save_path):
#             # Load existing matrix
#             existing_matrix = scipy.sparse.load_npz(save_path)

#             # Sum the new co-occurrence values with the existing ones
#             new_matrix = existing_matrix + new_matrix

#         # Save updated matrix
#         # print('Saved 10000 batches')
#         scipy.sparse.save_npz(save_path, new_matrix)

#         # Clear lists to free memory
#         word_ids_list.clear()
#         context_ids_list.clear()
#         values_list.clear()


# # Run computation on A100 GPU
# compute_cooccurrence(token_ids)

# # Load back the sparse matrix (Optional)
# loaded_matrix = scipy.sparse.load_npz(save_path)

# print(f"Co-occurrence matrix saved at: {save_path}")
# print(f"Non-zero co-occurrence pairs: {loaded_matrix.nnz}")

In [ ]:
# import torch
# import numpy as np
# import scipy.sparse
# import os
# from IPython.display import clear_output

# # File paths
# save_path = "/content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MB.npz"
# checkpoint_path = "/content/drive/MyDrive/DummyFolder/cooccurrence_checkpoint_50MB.txt"

# # Convert sentences to ID arrays
# token_ids = [[word_to_id[word] for word in sent] for sent in tokenized_sentences]
# token_ids = [torch.tensor(sent, dtype=torch.int32, device=device) for sent in token_ids]

# # Vocabulary size
# vocab_size = len(word_to_id)

# # Window size and batch size
# window_size = 8
# batch_size = 10000  # Adjust based on available memory

# # Initialize storage variables
# word_ids_list, context_ids_list, values_list = [], [], []

# # **Load checkpoint (if exists)**
# if os.path.exists(checkpoint_path):
#     with open(checkpoint_path, "r") as f:
#         last_processed_index = int(f.read().strip())
# else:
#     last_processed_index = 0

# # **Recovery Logic**
# def recover_cooccurrence_data(file_path):
#     """Attempt to recover data from a corrupted .npz file."""
#     try:
#         with np.load(file_path, allow_pickle=True) as recovered_data:
#             recovered_matrix = scipy.sparse.coo_matrix(
#                 (recovered_data['data'], (recovered_data['row'], recovered_data['col'])),
#                 shape=(vocab_size, vocab_size)
#             )
#             print("✅ Recovery successful!")
#             return recovered_matrix
#     except Exception as e:
#         print(f"❌ Recovery failed: {e}")
#         return scipy.sparse.coo_matrix((vocab_size, vocab_size))

# # Load the matrix (recovery or initialize)
# if os.path.exists(save_path):
#     existing_matrix = recover_cooccurrence_data(save_path)
# else:
#     existing_matrix = scipy.sparse.coo_matrix((vocab_size, vocab_size))

# def compute_cooccurrence(token_ids, start_index):
#     """Compute co-occurrence matrix using a sliding window approach."""
#     global word_ids_list, context_ids_list, values_list

#     for idx in range(start_index, len(token_ids)):
#         sentence = token_ids[idx]
#         sent_len = sentence.shape[0]

#         for i in range(sent_len):
#             word_id = sentence[i].item()
#             start = max(0, i - window_size)
#             end = min(sent_len, i + window_size + 1)

#             for j in range(start, end):
#                 if i != j:
#                     context_id = sentence[j].item()
#                     distance = abs(j - i)
#                     increment = 1.0 / distance
#                     word_ids_list.append(word_id)
#                     context_ids_list.append(context_id)
#                     values_list.append(increment)

#             # Save data when reaching batch size
#             if len(word_ids_list) >= batch_size:
#                 save_to_disk()

#         # Periodic flushing & checkpointing
#         if idx % 1000 == 0:
#             save_to_disk()
#             with open(checkpoint_path, "w") as f:
#                 f.write(str(idx))  # Save last processed index

#     # Final save & checkpoint update
#     save_to_disk()
#     with open(checkpoint_path, "w") as f:
#         f.write(str(len(token_ids)))  # Mark all sentences as processed

# def save_to_disk():
#     """Append co-occurrence data to an existing sparse matrix instead of overwriting."""
#     global word_ids_list, context_ids_list, values_list, existing_matrix

#     if word_ids_list:
#         # Convert current batch to sparse matrix
#         new_matrix = scipy.sparse.coo_matrix(
#             (values_list, (word_ids_list, context_ids_list)),
#             shape=(vocab_size, vocab_size)
#         )

#         # Sum the new co-occurrence values with the existing ones
#         existing_matrix += new_matrix

#         # Save updated matrix
#         clear_output()
#         print(f'Saved batch of {batch_size} words')
#         scipy.sparse.save_npz(save_path, existing_matrix)

#         # Clear lists to free memory
#         word_ids_list.clear()
#         context_ids_list.clear()
#         values_list.clear()

# # **Resume computation from last checkpoint**
# compute_cooccurrence(token_ids, last_processed_index)

# # Load back the sparse matrix (Optional)
# loaded_matrix = scipy.sparse.load_npz(save_path)

# print(f"Co-occurrence matrix saved at: {save_path}")
# print(f"Non-zero co-occurrence pairs: {loaded_matrix.nnz}")


In [35]:
import torch
import numpy as np
import scipy.sparse
import os

# Make sure device is set to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Vocabulary size and window size
vocab_size = len(word_to_id)
window_size = 8
token_ids = [[word_to_id[word] for word in sent] for sent in tokenized_sentences]

# Convert to PyTorch GPU tensors
token_ids = [torch.tensor(sent, dtype=torch.int32, device=device) for sent in token_ids]
# File path for saving the matrix
save_path = "/content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MBultimate.npz"

# Allocate a dense co-occurrence matrix on the GPU
# Note: This uses vocab_size x vocab_size elements. For a large vocabulary,
# ensure your GPU has enough memory.
cooccurrence_matrix = torch.zeros((vocab_size, vocab_size), dtype=torch.float32, device=device)

def compute_cooccurrence_gpu(token_ids):
    """Compute the co-occurrence matrix on the GPU using a dense matrix."""
    for idx, sentence in enumerate(token_ids):
        sent_len = sentence.shape[0]
        for i in range(sent_len):
            word_id = sentence[i].item()  # Get the current word's ID
            # Determine the window boundaries
            start = max(0, i - window_size)
            end = min(sent_len, i + window_size + 1)
            for j in range(start, end):
                if i != j:
                    context_id = sentence[j].item()
                    distance = abs(j - i)
                    increment = 1.0 / distance
                    cooccurrence_matrix[word_id, context_id] += increment

        # Print progress every 5000 sentences
        if idx % 5000 == 0:
            print(f"Processed {idx} sentences...")

    # Convert the dense GPU matrix to CPU and then to a sparse matrix
    cooccurrence_cpu = cooccurrence_matrix.cpu().numpy()
    sparse_matrix = scipy.sparse.coo_matrix(cooccurrence_cpu)
    scipy.sparse.save_npz(save_path, sparse_matrix)
    print(f"Final matrix saved at: {save_path}")

# Run computation using GPU-based token_ids
compute_cooccurrence_gpu(token_ids)

# (Optional) Load and check the sparse matrix
loaded_matrix = scipy.sparse.load_npz(save_path)
print(f"Non-zero co-occurrence pairs: {loaded_matrix.nnz}")


Processed 0 sentences...
Processed 5000 sentences...
Processed 10000 sentences...
Processed 15000 sentences...
Processed 20000 sentences...
Processed 25000 sentences...
Processed 30000 sentences...
Processed 35000 sentences...
Processed 40000 sentences...
Processed 45000 sentences...
Processed 50000 sentences...
Processed 55000 sentences...
Final matrix saved at: /content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MBultimate.npz
Non-zero co-occurrence pairs: 7559781


In [12]:
# Load back the sparse matrix (Optional)
import scipy.sparse
save_path = "/content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MBultimate.npz"

loaded_matrix = scipy.sparse.load_npz(save_path)

print(f"Co-occurrence matrix saved at: {save_path}")
print(f"Non-zero co-occurrence pairs: {loaded_matrix.nnz}")

Co-occurrence matrix saved at: /content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MBultimate.npz
Non-zero co-occurrence pairs: 7559781


In [13]:
import scipy.sparse
import numpy as np

# Load the sparse co-occurrence matrix
cooccur_sparse = scipy.sparse.load_npz("/content/drive/MyDrive/DummyFolder/cooccurrence_matrix_50MBultimate.npz")

# Convert to COO format (if not already)
cooccur_sparse = cooccur_sparse.tocoo()

# Convert sparse matrix to dictionary: {(word_id, context_id): co-occurrence_count}
cooccurrence = {
    (int(i), int(j)): float(value)
    for i, j, value in zip(cooccur_sparse.row, cooccur_sparse.col, cooccur_sparse.data)
}

# Print first 5 entries to verify
for k, v in list(cooccurrence.items())[:5]:
    print(f"{k} -> {v}")


(0, 0) -> 156.94528198242188
(0, 1) -> 134.8036346435547
(0, 2) -> 24.700002670288086
(0, 3) -> 14.749996185302734
(0, 4) -> 22.74166488647461


In [40]:
import gc

# Delete GPU tensors that are no longer needed.
del cooccurrence_matrix
del token_ids  # if you're done with these

# Run garbage collection
gc.collect()

# Clear PyTorch's cache
torch.cuda.empty_cache()

print("GPU memory has been flushed.")


GPU memory has been flushed.


In [ ]:
# total_values = cooccur_sparse.data.sum()
# print(f"Total co-occurrence values: {total_values}")
# unique_word_ids = np.unique(cooccur_sparse.row)
# total_word_ids = len(unique_word_ids)
# print(f"Total unique word IDs: {total_word_ids}")
# unique_context_ids = np.unique(cooccur_sparse.col)
# total_context_ids = len(unique_context_ids)
# print(f"Total unique context IDs: {total_context_ids}")
# total_nonzero_pairs = cooccur_sparse.nnz
# print(f"Total non-zero co-occurrence pairs: {total_nonzero_pairs}")


In [ ]:
# import scipy.sparse

# scipy.sparse.save_npz("/content/drive/MyDrive/DummyFolder/cooccurrence_matrix.npz", cooccur_sparse)

In [ ]:
# print(cooccurrence)
# print(cooccur_matrix)

In [ ]:
# print(np.sum(cooccur_matrix == 0) / np.prod(cooccur_matrix.shape))

# **GloVe Model Preparation and Execution**

**Initialization**

In [ ]:
# # Initialize GloVe Model Parameters

# embedding_dim = 50  # Dimensionality of embeddings
# # Word and context vectors (random initialization between -0.5 and 0.5)
# W = np.random.rand(vocab_size, embedding_dim) - 0.5
# W_context = np.random.rand(vocab_size, embedding_dim) - 0.5
# # Bias terms for words and contexts
# # bias = np.random.rand(vocab_size) - 0.5
# # bias_context = np.random.rand(vocab_size) - 0.5
# bias = np.zeros(vocab_size)
# bias_context = np.zeros(vocab_size)

In [ ]:
# print(bias)

In [ ]:
# print(W)

In [ ]:
# # Define Hyperparameters & Weighting Function

# epochs = 25
# learning_rate = 0.05
# x_max = 100  # Parameter for weighting function
# alpha = 0.75

# def weighting(x):
#     """Weighting function f(x) as used in the GloVe paper."""
#     return (x / x_max)**alpha if x < x_max else 1.0

**Training Glove Model using SGD**

In [ ]:
# # Training Loop using SGD

# for epoch in range(epochs):
#     total_loss = 0.0
#     # Loop over each non-zero co-occurrence pair
#     for (i, j), Xij in cooccurrence.items():
#         # Weight for this pair
#         weight = weighting(Xij)
#         # Compute the inner product between word and context vectors
#         dot = np.dot(W[i], W_context[j])
#         # Compute the error (difference from log co-occurrence count)
#         cost = dot + bias[i] + bias_context[j] - math.log(Xij)
#         # Weighted error
#         weighted_cost = weight * cost

#         # Accumulate loss (squared error)
#         total_loss += 0.5 * weighted_cost * cost

#         # Compute gradients
#         grad_wi = weighted_cost * W_context[j]
#         grad_wj_context = weighted_cost * W[i]

#         # Update the word and context vectors using SGD
#         W[i] -= learning_rate * grad_wi
#         W_context[j] -= learning_rate * grad_wj_context

#         # Update the biases
#         # bias[i] -= learning_rate * weighted_cost
#         # bias_context[j] -= learning_rate * weighted_cost

#         bias[i] -= (learning_rate * 0.1) * weighted_cost
#         bias_context[j] -= (learning_rate * 0.1) * weighted_cost

#     print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")

In [14]:
# import torch
# import math
# import scipy.sparse  # if needed elsewhere
# import numpy as np

# # Set device to GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Assume word_to_id and cooccurrence are already defined.
# # vocab_size is defined as:
# vocab_size = len(word_to_id)

# # GloVe hyperparameters
# embedding_dim = 50     # Dimensionality of embeddings
# epochs = 25
# learning_rate = 0.05
# x_max = 100.0          # For weighting function
# alpha = 0.75

# def weighting(x):
#     """Weighting function f(x) as used in the GloVe paper."""
#     return (x / x_max)**alpha if x < x_max else 1.0

# # Initialize model parameters as PyTorch tensors on the GPU.
# # We use torch.rand to initialize uniformly in [0,1) then subtract 0.5.
# W = torch.rand(vocab_size, embedding_dim, device=device) - 0.5
# W_context = torch.rand(vocab_size, embedding_dim, device=device) - 0.5
# bias = torch.zeros(vocab_size, device=device)
# bias_context = torch.zeros(vocab_size, device=device)

# # Training loop using SGD (manual updates)
# for epoch in range(epochs):
#     total_loss = 0.0
#     # Iterate over each non-zero co-occurrence pair in the dictionary.
#     # cooccurrence is assumed to be a dict: {(i, j): Xij}
#     for (i, j), Xij in cooccurrence.items():
#         weight = weighting(Xij)

#         # Compute the inner product between the word vector and context vector.
#         dot = torch.dot(W[i], W_context[j])

#         # Compute the error from the log co-occurrence count.
#         cost = dot + bias[i] + bias_context[j] - torch.log(torch.tensor(float(Xij), device=device))
#         weighted_cost = weight * cost

#         # Accumulate loss (convert the scalar to a Python float)
#         total_loss += 0.5 * (weighted_cost * cost).item()

#         # Compute gradients (all operations occur on GPU)
#         grad_wi = weighted_cost * W_context[j]
#         grad_wj_context = weighted_cost * W[i]

#         # Update parameters using SGD
#         W[i] = W[i] - learning_rate * grad_wi
#         W_context[j] = W_context[j] - learning_rate * grad_wj_context
#         bias[i] = bias[i] - (learning_rate * 0.1) * weighted_cost
#         bias_context[j] = bias_context[j] - (learning_rate * 0.1) * weighted_cost

#     print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")


KeyboardInterrupt: 

In [120]:
import torch
import math
import numpy as np

# Set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume word_to_id is already defined and cooccurrence is a dict: {(i, j): Xij}
vocab_size = len(word_to_id)

# GloVe hyperparameters
embedding_dim = 150     # Dimensionality of embeddings
epochs = 150
learning_rate = 0.05
x_max = 100.0          # For weighting function
alpha = 0.75

def weighting(x):
    """Vectorized weighting function f(x) as used in the GloVe paper."""
    return torch.where(x < x_max, (x / x_max)**alpha, torch.ones_like(x))

# Initialize model parameters as PyTorch tensors on the GPU.
W = torch.rand(vocab_size, embedding_dim, device=device) - 0.5
W_context = torch.rand(vocab_size, embedding_dim, device=device) - 0.5
bias = torch.zeros(vocab_size, device=device)
bias_context = torch.zeros(vocab_size, device=device)

# --- Preprocess co-occurrence dictionary into tensors ---
# Convert dictionary keys and values to lists and then to tensors.
pairs = list(cooccurrence.keys())           # List of tuples (i, j)
values = list(cooccurrence.values())          # List of Xij values

# Convert to tensors (on GPU)
pairs_tensor = torch.tensor(pairs, dtype=torch.long, device=device)  # shape (num_pairs, 2)
Xij_tensor = torch.tensor(values, dtype=torch.float32, device=device)  # shape (num_pairs,)

# Pre-compute log(Xij)
log_Xij_tensor = torch.log(Xij_tensor)

num_pairs = pairs_tensor.shape[0]
batch_size = 100000  # Adjust batch size based on your GPU memory

# --- Vectorized Training Loop ---
for epoch in range(epochs):
    total_loss = 0.0

    # Shuffle indices for each epoch
    perm = torch.randperm(num_pairs, device=device)
    pairs_shuffled = pairs_tensor[perm]
    Xij_shuffled = Xij_tensor[perm]
    log_Xij_shuffled = log_Xij_tensor[perm]

    # Process in batches
    for start in range(0, num_pairs, batch_size):
        end = min(start + batch_size, num_pairs)
        batch_pairs = pairs_shuffled[start:end]
        batch_Xij = Xij_shuffled[start:end]
        batch_log_Xij = log_Xij_shuffled[start:end]

        # Separate word indices and context indices
        i_batch = batch_pairs[:, 0]
        j_batch = batch_pairs[:, 1]

        # Vectorized dot product for batch: (batch_size,)
        dot = torch.sum(W[i_batch] * W_context[j_batch], dim=1)

        # Compute cost for the batch
        cost = dot + bias[i_batch] + bias_context[j_batch] - batch_log_Xij
        weight = weighting(batch_Xij)
        weighted_cost = weight * cost

        # Accumulate loss
        loss = 0.5 * torch.sum(weighted_cost * cost)
        total_loss += loss.item()

        # Compute gradients for batch (each is (batch_size, embedding_dim))
        grad_wi = weighted_cost.unsqueeze(1) * W_context[j_batch]
        grad_wj_context = weighted_cost.unsqueeze(1) * W[i_batch]

        # Now update parameters using scatter-add style accumulation.
        # Using index_put_ with accumulate=True applies the updates to repeated indices.
        W.index_put_((i_batch,), -learning_rate * grad_wi, accumulate=True)
        W_context.index_put_((j_batch,), -learning_rate * grad_wj_context, accumulate=True)
        bias.index_put_((i_batch,), -learning_rate * 0.1 * weighted_cost, accumulate=True)
        bias_context.index_put_((j_batch,), -learning_rate * 0.1 * weighted_cost, accumulate=True)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")


Epoch 1/150, Loss: 359538.4501953125
Epoch 2/150, Loss: 205648.4522705078
Epoch 3/150, Loss: 173963.54553222656
Epoch 4/150, Loss: 155303.9610595703
Epoch 5/150, Loss: 142313.28381347656
Epoch 6/150, Loss: 132445.70178222656
Epoch 7/150, Loss: 124488.76147460938
Epoch 8/150, Loss: 117904.81372070312
Epoch 9/150, Loss: 112364.63482666016
Epoch 10/150, Loss: 107651.12829589844
Epoch 11/150, Loss: 103459.58471679688
Epoch 12/150, Loss: 99780.87908935547
Epoch 13/150, Loss: 96502.45373535156
Epoch 14/150, Loss: 93519.30456542969
Epoch 15/150, Loss: 90862.00756835938
Epoch 16/150, Loss: 88392.67272949219
Epoch 17/150, Loss: 86139.39196777344
Epoch 18/150, Loss: 84067.23706054688
Epoch 19/150, Loss: 82139.38116455078
Epoch 20/150, Loss: 80385.11706542969
Epoch 21/150, Loss: 78688.20849609375
Epoch 22/150, Loss: 77137.19500732422
Epoch 23/150, Loss: 75648.0005493164
Epoch 24/150, Loss: 74310.01959228516
Epoch 25/150, Loss: 72967.24151611328
Epoch 26/150, Loss: 71778.56427001953
Epoch 27/150, 

***Normal (Gaussian) initialization***

In [ ]:
# import torch
# import math
# import numpy as np

# # Set device to GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Assume word_to_id is already defined and cooccurrence is a dict: {(i, j): Xij}
# vocab_size = len(word_to_id)

# # GloVe hyperparameters
# embedding_dim = 150     # Dimensionality of embeddings
# epochs = 100
# learning_rate = 0.05
# x_max = 100.0          # For weighting function
# alpha = 0.75

# def weighting(x):
#     """Vectorized weighting function f(x) as used in the GloVe paper."""
#     return torch.where(x < x_max, (x / x_max)**alpha, torch.ones_like(x))

# # Initialize model parameters as PyTorch tensors on the GPU.

# # Create empty tensors for word embeddings and context embeddings.
# W = torch.empty(vocab_size, embedding_dim, device=device)
# W_context = torch.empty(vocab_size, embedding_dim, device=device)

# # Apply Normal (Gaussian) initialization with mean=0.0 and std=0.01
# torch.nn.init.normal_(W, mean=0.0, std=0.01)
# torch.nn.init.normal_(W_context, mean=0.0, std=0.01)

# # Initialize bias terms (commonly to zeros)
# bias = torch.zeros(vocab_size, device=device)
# bias_context = torch.zeros(vocab_size, device=device)

# print("Normal (Gaussian) initialization complete for W and W_context.")

# # --- Preprocess co-occurrence dictionary into tensors ---
# # Convert dictionary keys and values to lists and then to tensors.
# pairs = list(cooccurrence.keys())           # List of tuples (i, j)
# values = list(cooccurrence.values())          # List of Xij values

# # Convert to tensors (on GPU)
# pairs_tensor = torch.tensor(pairs, dtype=torch.long, device=device)  # shape (num_pairs, 2)
# Xij_tensor = torch.tensor(values, dtype=torch.float32, device=device)  # shape (num_pairs,)

# # Pre-compute log(Xij)
# log_Xij_tensor = torch.log(Xij_tensor)

# num_pairs = pairs_tensor.shape[0]
# batch_size = 100000  # Adjust batch size based on your GPU memory

# # --- Vectorized Training Loop ---
# for epoch in range(epochs):
#     total_loss = 0.0

#     # Shuffle indices for each epoch
#     perm = torch.randperm(num_pairs, device=device)
#     pairs_shuffled = pairs_tensor[perm]
#     Xij_shuffled = Xij_tensor[perm]
#     log_Xij_shuffled = log_Xij_tensor[perm]

#     # Process in batches
#     for start in range(0, num_pairs, batch_size):
#         end = min(start + batch_size, num_pairs)
#         batch_pairs = pairs_shuffled[start:end]
#         batch_Xij = Xij_shuffled[start:end]
#         batch_log_Xij = log_Xij_shuffled[start:end]

#         # Separate word indices and context indices
#         i_batch = batch_pairs[:, 0]
#         j_batch = batch_pairs[:, 1]

#         # Vectorized dot product for batch: (batch_size,)
#         dot = torch.sum(W[i_batch] * W_context[j_batch], dim=1)

#         # Compute cost for the batch
#         cost = dot + bias[i_batch] + bias_context[j_batch] - batch_log_Xij
#         weight = weighting(batch_Xij)
#         weighted_cost = weight * cost

#         # Accumulate loss
#         loss = 0.5 * torch.sum(weighted_cost * cost)
#         total_loss += loss.item()

#         # Compute gradients for batch (each is (batch_size, embedding_dim))
#         grad_wi = weighted_cost.unsqueeze(1) * W_context[j_batch]
#         grad_wj_context = weighted_cost.unsqueeze(1) * W[i_batch]

#         # Now update parameters using scatter-add style accumulation.
#         # Using index_put_ with accumulate=True applies the updates to repeated indices.
#         W.index_put_((i_batch,), -learning_rate * grad_wi, accumulate=True)
#         W_context.index_put_((j_batch,), -learning_rate * grad_wj_context, accumulate=True)
#         bias.index_put_((i_batch,), -learning_rate * 0.1 * weighted_cost, accumulate=True)
#         bias_context.index_put_((j_batch,), -learning_rate * 0.1 * weighted_cost, accumulate=True)

#     print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")


**Save Generated Embeddings in txt Format**

In [ ]:
# # Combine and Save the Final Embeddings in txt format

# # A common practice is to sum (or average) the word and context embeddings
# final_embeddings = W + W_context

# # Save embeddings to a text file (each line: word followed by its embedding)
# with open('/content/drive/MyDrive/DummyFolder/HNVec100MB.txt', 'w', encoding='utf-8') as f:
#     for word, i in word_to_id.items():
#         vector = ' '.join(map(str, final_embeddings[i]))
#         f.write(f"{word} {vector}\n")

# print("Training complete. Embeddings saved to 'HNVec.txt'.")

In [121]:
# Combine and Save the Final Embeddings in txt format

# A common practice is to sum (or average) the word and context embeddings
final_embeddings = (W + W_context).cpu()  # Move to CPU

# Save embeddings to a text file (each line: word followed by its embedding)
output_file = '/content/drive/MyDrive/DummyFolder/HNVec50MB.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    for word, i in word_to_id.items():
        # Convert tensor to list for cleaner formatting
        vector = ' '.join(map(str, final_embeddings[i].tolist()))
        f.write(f"{word} {vector}\n")

print(f"Training complete. Embeddings saved to '{output_file}'.")


Training complete. Embeddings saved to '/content/drive/MyDrive/DummyFolder/HNVec50MB.txt'.


**Save Glove model in pkl format**

In [ ]:
# import pickle

# # Save the embeddings dictionary as a .pkl file
# model_data = {
#     "word_to_id": word_to_id,
#     "id_to_word": {i: word for word, i in word_to_id.items()},
#     "embeddings": final_embeddings
# }

# with open("/content/drive/MyDrive/DummyFolder/HNVec100MB.pkl", "wb") as f:
#     pickle.dump(model_data, f)

# print("Model saved as 'HNVec100MB.pkl'.")


In [130]:
import pickle

# (Optional) Ensure final_embeddings is on CPU and convert to numpy if it's a tensor.
final_embeddings = final_embeddings.cpu().detach().numpy()

# Save the embeddings dictionary as a .pkl file
model_data = {
    "word_to_id": word_to_id,
    "id_to_word": {i: word for word, i in word_to_id.items()},
    "embeddings": final_embeddings
}

model_output = '/content/drive/MyDrive/DummyFolder/HNVec50MB.pkl'
with open(model_output, "wb") as f:
    pickle.dump(model_data, f)

print(f"Model saved to '{model_output}'.")


Model saved to '/content/drive/MyDrive/DummyFolder/HNVec50MB.pkl'.


# **Steps to import and use Glove Model**

In [131]:
# Steps to import the model and get corresponding vector embedding using pkl

import pickle
import numpy as np

# Load the saved model
with open("/content/drive/MyDrive/DummyFolder/HNVec50MB.pkl", "rb") as f:
    model_data = pickle.load(f)

# Extract components
word_to_id = model_data["word_to_id"]
id_to_word = model_data["id_to_word"]
embeddings = model_data["embeddings"]

# Example: Get the embedding of a word
word = "मेट्रो"
if word in word_to_id:
    word_embedding = embeddings[word_to_id[word]]
    print(f"Embedding for '{word}': {word_embedding}")
else:
    print(f"Word '{word}' not found in vocabulary.")


Embedding for 'मेट्रो': [ 1.1470363   0.3918727  -0.07192533 -0.39991024 -0.9962837   0.16012998
  0.03748996  0.67268586  0.6152403  -0.16711852 -0.5713951   0.2055284
 -0.39232767  0.40831295  0.31375968 -0.25927767 -0.5682203   0.23190357
 -1.0165441  -0.13586225  0.07439667 -0.22753613 -0.5519527  -0.2954046
  0.65655345  0.59378463  0.6932719   0.19834243 -0.08218037 -0.4319578
  0.35499805  1.2892915   0.27267823  0.1628054   0.15434489  0.37593234
 -0.07960588  0.26832932  0.61413467 -0.41428658  0.33253273 -0.14717603
 -0.43292356 -0.5595272   0.8874601  -0.37730634  0.6848813   0.41775966
 -0.34516618 -0.3525604  -0.21011913 -0.17072406  0.8410872   0.11959081
  0.40918326 -0.81320655  0.7277419  -0.02005552 -0.8379956  -0.46914095
  0.5946795  -0.81984437 -0.06028949  0.16553348  0.18494162  0.03038067
 -0.6446216   0.4227677  -0.05241515 -0.3086492   0.38709456  0.4338052
  0.06849141 -0.32135004  0.06714053 -0.9235772  -0.2941364  -0.24309906
  0.00329646  0.41395935 -0.287

In [132]:
# Steps to get corresponding vector embedding using txt file
word_embeddings = {}
with open("/content/drive/MyDrive/DummyFolder/HNVec50MB.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        word_embeddings[word] = vector

# Get vector for a word
demo_word = "मेट्रो"
print(f"Embedding for '{demo_word}': {word_embeddings[demo_word]}")
# print(word_embeddings["भारत"])  # Example vector for "भारत"

Embedding for 'मेट्रो': [ 1.1470363   0.3918727  -0.07192533 -0.39991024 -0.9962837   0.16012998
  0.03748996  0.67268586  0.6152403  -0.16711852 -0.5713951   0.2055284
 -0.39232767  0.40831295  0.31375968 -0.25927767 -0.5682203   0.23190357
 -1.0165441  -0.13586225  0.07439667 -0.22753613 -0.5519527  -0.2954046
  0.65655345  0.59378463  0.6932719   0.19834243 -0.08218037 -0.4319578
  0.35499805  1.2892915   0.27267823  0.1628054   0.15434489  0.37593234
 -0.07960588  0.26832932  0.61413467 -0.41428658  0.33253273 -0.14717603
 -0.43292356 -0.5595272   0.8874601  -0.37730634  0.6848813   0.41775966
 -0.34516618 -0.3525604  -0.21011913 -0.17072406  0.8410872   0.11959081
  0.40918326 -0.81320655  0.7277419  -0.02005552 -0.8379956  -0.46914095
  0.5946795  -0.81984437 -0.06028949  0.16553348  0.18494162  0.03038067
 -0.6446216   0.4227677  -0.05241515 -0.3086492   0.38709456  0.4338052
  0.06849141 -0.32135004  0.06714053 -0.9235772  -0.2941364  -0.24309906
  0.00329646  0.41395935 -0.287

In [129]:
# Steps to find similar words
# We can find similar words using cosine similarity.
from sklearn.metrics.pairwise import cosine_similarity

def find_similar(word, top_n=5):
    if word not in word_embeddings:
        return "Word not in vocabulary"

    word_vec = word_embeddings[word].reshape(1, -1)
    similarities = {w: cosine_similarity(word_vec, v.reshape(1, -1))[0][0]
                    for w, v in word_embeddings.items()}
    sorted_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[1:top_n+1]
    return sorted_words

print(find_similar("मेट्रो"))


[('दिल्ली', 0.44579363), ('डांस', 0.43859777), ('रेल', 0.36082327), ('सफर', 0.35479024), ('कियाशानदार', 0.34631544)]


In [ ]:
# print(W)

In [133]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two vectors."""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def word_analogy(word_a, word_b, word_c, final_embeddings, word_to_id, id_to_word, top_n=5):
    """
    Solves the analogy: word_a is to word_b as word_c is to ?

    For example, if word_a = "king", word_b = "queen", word_c = "man",
    then the function should ideally return "woman" among the top results.
    """
    # Ensure all words are in the vocabulary.
    for word in [word_a, word_b, word_c]:
        if word not in word_to_id:
            print(f"Word '{word}' not found in the vocabulary.")
            return None

    # Compute the target vector: vec(word_b) - vec(word_a) + vec(word_c)
    vec_a = final_embeddings[word_to_id[word_a]]
    vec_b = final_embeddings[word_to_id[word_b]]
    vec_c = final_embeddings[word_to_id[word_c]]
    target_vec = vec_b - vec_a + vec_c

    # Normalize the target vector.
    target_vec = target_vec / np.linalg.norm(target_vec)

    # Compute cosine similarity with all words in the vocabulary.
    similarities = {}
    for word, idx in word_to_id.items():
        # Optionally skip the input words
        if word in [word_a, word_b, word_c]:
            continue
        vec = final_embeddings[idx]
        vec = vec / np.linalg.norm(vec)  # normalize
        sim = np.dot(target_vec, vec)
        similarities[word] = sim

    # Get the top N words sorted by similarity (highest first)
    sorted_sim = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return sorted_sim[:top_n]

# Example usage:
# (Assume final_embeddings is already on CPU and in NumPy array format)
# For example, let's say we want to test the analogy:
# "भारतीय" is to "अंतरिक्ष" as "SomeOtherWord" is to ?
# Replace 'SomeOtherWord' with a word that is in your vocabulary.

analogy_result = word_analogy("राजा", "रानी", "पुरुष" ,
                               final_embeddings, word_to_id, id_to_word, top_n=5)
if analogy_result:
    print("Analogy results (word, similarity):")
    for word, sim in analogy_result:
        print(f"{word}: {sim:.4f}")


Analogy results (word, similarity):
हॉकी: 0.3599
महिला: 0.3387
वेस्टइंडीज: 0.3266
संबंध: 0.3212
आबेदीन: 0.3152
